In [1]:
# import json
#
# import rel
# import websocket
from PyQt6.QtGui import QFontDatabase
# from hearthstone.enums import BlockType, Zone, Step, PlayState

from Coach import Coach
from Game import GameImp as Game
from NN import NNetWrapper as nn
# from fireplace.actions import Attack, Summon, Hit, EndTurn, Discover, Choice, MulliganChoice, Play, GenericChoice, \
#     BeginTurn, Death, TargetedAction, Activate
# from fireplace.card import HeroPower, Hero, Character
# from fireplace.exceptions import GameOver
# from fireplace.managers import BaseObserver
# from fireplace.player import Player
from ui.ui import MainWindow
from PyQt6.QtWidgets import *
import sys
import re
import os
import time
import math

import torch
import torch.utils.data
from observers import UiObserver, HsObserver

PATH TO CARDS = ui\cards


In [2]:
class Args:
    def __init__(self):
        self.n_in = 34 * 16
        self.n_out = 21 * 18

        self.epochs = 15
        self.batch_size = 128
        self.lr = 1e-4

        self.arenaCompare = 10
        self.numIters = 30
        self.numEps = 13
        self.maxlenOfQueue = 100000
        self.numMCTS = 700
        self.numItersForTrainExamplesHistory = 20

        self.tempThreshold = 15
        self.updateThreshold = 0.6

        self.ngpu = 0
        print("cuda:0" if (torch.cuda.is_available() and self.ngpu > 0) else "cpu")
        self.device = torch.device("cuda:0" if (torch.cuda.is_available() and self.ngpu > 0) else "cpu")

        self.checkpoint = './temp/'
        self.load_model = True
        self.load_folder_file = ('./temp/', 'temp.pth.tar')
        self.load_examples = ('./temp/', 'checkpoint.pth.tar')
        self.fireplace_log_enabled = True

In [ ]:
args = Args()
g = Game()
nnet = nn(args)

if args.load_model:
    nnet.load_checkpoint(args.load_folder_file[0], args.load_folder_file[1])

c = Coach(g, nnet, args)
if args.load_model:
    print("Load trainExamples from file")
    c.load_train_examples()
c.learn()